# Inspecting pickled `exactly` hits

## Imports

In [1]:
import re
from pathlib import Path

import pandas as pd

EXACTLY_OUT = Path('/share/compling/projects/sanpi/notebooks/exactly_out')

### ~~Define~~ Import helper `utils`

~~Copied from `./source/analyze/utils/`~~\
~~jupyter won't import them 🤷‍♀️~~

In [2]:
from source.utils.dataframes import (balance_sample, cols_by_str, concat_pkls,
                                     make_cats, optimize_hit_df, show_counts,
                                     summarize_text_cols, corners)
from source.utils.general import find_files, print_iter
from source.utils.sample import sample_pickle
from source.utils.visualize import heatmap

In [3]:
# def find_files(data_dir: Path, fname_glob: str, verbose: bool = False):
    # path_iter = data_dir.rglob(fname_glob)
    # if verbose:
    #     path_iter = tuple(path_iter)
    #     print_iter(
    #         [f'../{p.relative_to(data_dir)}' for p in path_iter], bullet='-',
    #         header=f'### {len(path_iter)} paths matching {fname_glob} found in {data_dir}')
    # return path_iter


# def print_iter(iter_obj,
    #            bullet: str = '▸',
    #            # //    logger: logging.Logger = None,
    #            # //    level: int = 20,
    #            header: str = ''):

    # bullet_str = f'\n{bullet} '

    # iter_str = bullet_str.join(f'{i}' for i in iter_obj)

    # msg_str = f'\n{header}{bullet_str}{iter_str}'
    # msg_str = msg_str.replace('\n\n', '\n').strip(f'{bullet} ')

    # print(msg_str)


# def cols_by_str(df: pd.DataFrame, start_str=None, end_str=None) -> list:
    # if end_str:
    #     cols = df.columns[df.columns.str.endswith(end_str)]
    #     if start_str:
    #         cols = cols[cols.str.startswith(start_str)]
    # elif start_str:
    #     cols = df.columns[df.columns.str.startswith(start_str)]
    # else:
    #     cols = df.columns

    # return cols.to_list()


# def make_cats(orig_df: pd.DataFrame, columns: list = None) -> pd.DataFrame:
    # df = orig_df.copy()
    # if columns is None:
    #     cat_suff = ("code", "name", "path", "stem")
    #     columns = df.columns.str.endswith(cat_suff)  # type: ignore
    # df.loc[:, columns] = df.loc[:, columns].astype(
    #     'string').fillna('_').astype('category')

    # return df


# def balance_sample(full_df: pd.DataFrame,
    #                column_name: str = 'category',
    #                sample_per_value: int = 5,
    #                verbose: bool = False) -> tuple:
    # '''
    # create sample with no more than n rows satisfying each unique value
    # of the given column. A value of -1 for `sample_per_value` will limit
    # all values' results to the minimum count per value.
    # '''
    # info_message = ''
    # subsamples = []
    # for __, col_val_df in full_df.groupby(column_name):
    #     # take sample if 1+ and less than length of full dataframe
    #     if len(col_val_df) > sample_per_value > 0:
    #         subsample_df = col_val_df.sample(sample_per_value)
    #         subsamples.append(subsample_df)
    #     else:
    #         subsamples.append(col_val_df)

    # # > trim all "by column" sub dfs to length of shortest if -1 given
    # if sample_per_value == -1:
    #     trim_len = int(min(len(sdf) for sdf in subsamples))
    #     subsamples = [sdf.sample(trim_len)
    #                   for sdf in subsamples]

    # b_sample = pd.concat(subsamples)

    # if verbose:
    #     subset_info_table = (
    #         b_sample
    #         .value_counts(subset=column_name)
    #         .to_frame(name='count')
    #         .assign(percentage=b_sample
    #                 .value_counts(column_name, normalize=True)
    #                 .round(2) * 100)
    #         .to_markdown())
    #     label = (full_df.hits_df_pkl[0].stem + ' '
    #              if 'hits_df_pkl' in full_df.columns
    #              else '')
    #     info_message = (f'\n## {column_name} representation in {label}sample\n'
    #                     + subset_info_table)

    # return b_sample, info_message


# def concat_pkls(data_dir: Path = Path('/share/compling/data/sanpi/2_hit_tables'),
    #             fname_glob: str = '*.pkl.gz',
    #             pickles=None,
    #             convert_dtypes=False,
    #             verbose: bool = True) -> pd.DataFrame:
    # if not pickles:
    #     pickles = find_files(Path(data_dir), fname_glob, verbose)

    # # tested and found that it is faster to assign `corpus` intermittently
    # df = pd.concat((pd.read_pickle(p).assign(corpus=p.stem.rsplit('_', 2)[0])
    #                 for p in pickles))

    # dup_check_cols = cols_by_str(df, end_str=('text', 'id', 'sent'))
    # df = (df.loc[~df.duplicated(subset=dup_check_cols), :])
    # df = df.convert_dtypes()
    # df = make_cats(df, (['corpus'] + cols_by_str(df, start_str=('nr', 'neg', 'adv'),
    #                                              end_str=('lemma', 'form'))))

    # return df
# def _optimize_df(df: pd.DataFrame) -> pd.DataFrame:

    # # print('Original Dataframe:')
    # # df.info(memory_usage='deep')

    # # > drop unneeded string columns
    # # was:
    # #   for c in udf.cols_by_str(df, start_str=('context', 'text', 'sent_text', 'token')):
    # for c in cols_by_str(df, start_str=('context', 'token', 'utt')):
    #     df.pop(c)

    # # > select only non-`object` dtype columns
    # relevant_cols = df.columns[~df.dtypes.astype(
    #     'string').str.endswith(('object'))]
    # # limit df to `relevant_cols`
    # df = df[relevant_cols]

    # # > create empty dataframe with `relevant_cols` as index/rows
    # df_info = pd.DataFrame(index=relevant_cols)

    # df_info = df_info.assign(
    #     mem0=df.memory_usage(deep=True),
    #     dtype0=df.dtypes.astype('string'),
    #     defined_values=df.count(),
    #     unique_values=df.apply(pd.unique, axis=0).apply(len))
    # df_info = df_info.assign(
    #     ratio_unique=(df_info.unique_values/df_info.defined_values).round(2))

    # cat_candidates = df_info.loc[df_info.ratio_unique < 0.8,
    #                              :].loc[df_info.dtype0 != 'category'].index.to_list()
    # #was: catted_df = udf.make_cats(df.copy(), cat_candidates)
    # catted_df = make_cats(df.copy(), cat_candidates)

    # df_info = df_info.assign(dtype1=catted_df.dtypes,
    #                          mem1=catted_df.memory_usage(deep=True))
    # df_info = df_info.assign(mem_change=df_info.mem1-df_info.mem0)
    # print(df_info.sort_values(
    #     ['mem_change', 'ratio_unique', 'dtype0']).to_markdown())
    # mem_improved = df_info.loc[df_info.mem_change < 0, :].index.to_list()
    # for c in df.columns[~df.columns.isin(mem_improved)]:
    #     print(c, '\t', df.loc[:, c].dtype)
    # df.loc[:, mem_improved] = catted_df.loc[:, mem_improved]
    # print('Category Converted dataframe:')
    # df.info(memory_usage='deep')

    # return df

### ~~Newly created~~ Moved to [`source.utils.dataframes`](source/utils/dataframes.py)

In [4]:
# * moved to to utils.dataframes
# def show_counts(df, columns):
    # return df.value_counts(columns).to_frame().rename(columns={0: 'count'})


# def summarize_text_cols(tdf: pd.DataFrame):

    # summary = tdf.describe().transpose()
    # summary = summary.assign(top_percent=(
    #     ((pd.to_numeric(summary.freq) / len(tdf)))*100).round(2))
    # summary = summary.rename(columns={'top': 'top_value', 'freq': 'top_freq'})

    # return summary.convert_dtypes().sort_values('unique')

## Load Data

In [5]:
ddf = concat_pkls(
    data_dir=Path('/share/compling/data/sanpi/3_dep_info'),
    fname_glob='exactly*hits+deps.pkl.gz',
    convert_dtypes=True)


### 12 paths matching exactly*hits+deps.pkl.gz found in /share/compling/data/sanpi/3_dep_info
- ../raised/exactly_apw_neg-raised_hits+deps.pkl.gz
- ../raised/exactly_nyt_neg-raised_hits+deps.pkl.gz
- ../raised/exactly_puddin_neg-raised_hits+deps.pkl.gz
- ../scoped/exactly_apw_with-relay_hits+deps.pkl.gz
- ../scoped/exactly_nyt_with-relay_hits+deps.pkl.gz
- ../scoped/exactly_puddin_with-relay_hits+deps.pkl.gz
- ../contig/exactly_apw_sans-relay_hits+deps.pkl.gz
- ../contig/exactly_nyt_sans-relay_hits+deps.pkl.gz
- ../contig/exactly_puddin_sans-relay_hits+deps.pkl.gz
- ../advadj/exactly_apw_all-RB-JJs_hits+deps.pkl.gz
- ../advadj/exactly_nyt_all-RB-JJs_hits+deps.pkl.gz
- ../advadj/exactly_puddin_all-RB-JJs_hits+deps.pkl.gz


### Optimize DataFrame

In [6]:
odf = optimize_hit_df(ddf, verbosity=2)
odf

Original Dataframe:
<class 'pandas.core.frame.DataFrame'>
Index: 150437 entries, apw_eng_20030918_0697_20:4-5-8-9 to pcc_eng_09_047.0803_x0745587_19:3-4
Data columns (total 43 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   colloc             150437 non-null  string  
 1   sent_text          150437 non-null  string  
 2   nr_form            457 non-null     category
 3   neg_form           51496 non-null   category
 4   adv_form           150437 non-null  category
 5   adj_form           150437 non-null  string  
 6   hit_text           150437 non-null  string  
 7   text_window        150437 non-null  string  
 8   sent_id            150437 non-null  string  
 9   match_id           150437 non-null  string  
 10  colloc_id          150437 non-null  string  
 11  token_str          150437 non-null  string  
 12  lemma_str          150437 non-null  string  
 13  context_prev_id    150437 non-null  string  
 14  context_p

,colloc,nr_form,neg_form,adv_form,adj_form,hit_text,sent_id,match_id,colloc_id,lemma_str,...,dep_str_ix,dep_str_full,dep_str_rel,dep_str_mask,dep_str_mask_rel,corpus,relay_form,relay_lemma,relay_index,dep_relay
hit_id,,,,,,,,,,,,,,,,,,,,,
apw_eng_20030918_0697_20:4-5-8-9,exactly_safe,think,n't,exactly,safe,n't think it was exactly safe,apw_eng_20030918_0697_20,4-5-8-9,apw_eng_20030918_0697_20:8-9,`` I do not think it be exactly safe moviemake...,...,04:think>03:not[=neg]; 08:safe>07:exactly[=mod...,04:think>[neg]>03:not[=neg]; 08:safe>[advmod]>...,think>[neg]>not[=neg]; safe>[advmod]>exactly[=...,VB>not; JJ>RB; think>JJ,VB>[neg]>not; JJ>[advmod]>RB; think>[ccomp]>JJ,exactly_apw,<NA>,<NA>,NaN,NaN
apw_eng_20020929_0547_33:3-4-7-8,exactly_clear,think,n't,exactly,clear,n't think they 're exactly clear,apw_eng_20020929_0547_33,3-4-7-8,apw_eng_20020929_0547_33:7-8,I do not think they be exactly clear how to do...,...,03:think>02:not[=neg]; 07:clear>06:exactly[=mo...,03:think>[neg]>02:not[=neg]; 07:clear>[advmod]...,think>[neg]>not[=neg]; clear>[advmod]>exactly[...,VB>not; JJ>RB; think>JJ,VB>[neg]>not; JJ>[advmod]>RB; think>[ccomp]>JJ,exactly_apw,<NA>,<NA>,NaN,NaN
apw_eng_20020206_1493_20:4-5-8-9,exactly_correct,think,not,exactly,correct,not think that is exactly correct,apw_eng_20020206_1493_20,4-5-8-9,apw_eng_20020206_1493_20:8-9,"`` I do not think that be exactly correct , ''...",...,04:think>03:not[=neg]; 08:correct>07:exactly[=...,04:think>[neg]>03:not[=neg]; 08:correct>[advmo...,think>[neg]>not[=neg]; correct>[advmod]>exactl...,VB>not; JJ>RB; think>JJ,VB>[neg]>not; JJ>[advmod]>RB; think>[ccomp]>JJ,exactly_apw,<NA>,<NA>,NaN,NaN
apw_eng_19980320_1318_52:03-04-18-19,exactly_high,want,n't,exactly,high,"n't want to sound racist , but the intellectua...",apw_eng_19980320_1318_52,03-04-18-19,apw_eng_19980320_1318_52:18-19,"I do not want to sound racist , but the intell...",...,03:want>02:not[=neg]; 18:high>17:exactly[=mod]...,03:want>[neg]>02:not[=neg]; 18:high>[advmod]>1...,want>[neg]>not[=neg]; high>[advmod]>exactly[=m...,VB>not; JJ>RB; want>JJ,VB>[neg]>not; JJ>[advmod]>RB; want>[conj]>JJ,exactly_apw,<NA>,<NA>,NaN,NaN
nyt_eng_20091024_0144_21:04-05-10-11,exactly_sure,think,n't,exactly,sure,n't think any of us are exactly sure,nyt_eng_20091024_0144_21,04-05-10-11,nyt_eng_20091024_0144_21:10-11,`` I do not think any of we be exactly sure wh...,...,04:think>03:not[=neg]; 10:sure>09:exactly[=mod...,04:think>[neg]>03:not[=neg]; 10:sure>[advmod]>...,think>[neg]>not[=neg]; sure>[advmod]>exactly[=...,VB>not; JJ>RB; think>JJ,VB>[neg]>not; JJ>[advmod]>RB; think>[ccomp]>JJ,exactly_nyt,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pcc_eng_09_047.1818_x0747142_10:4-5,exactly_new,<NA>,<NA>,exactly,new,exactly new,pcc_eng_09_047.1818_x0747142_10,4-5,pcc_eng_09_047.1818_x0747142_10:4-5,which be not exactly new for a company that be...,...,04:new>03:exactly[=mod],04:new>[advmod]>03:exactly[=mod],new>[advmod]>exactly[=mod],JJ>RB,JJ>[advmod]>RB,exactly_puddin,<NA>,<NA>,NaN,NaN
pcc_eng_09_047.1686_x0746931_024:19-20,exactly_sure,<NA>,<NA>,exactly,sure,exactly sure,pcc_eng_09_047.1686_x0746931_024,19-20,pcc_eng_09_047.1686_x0746931_024:19-20,it may have have something to do with the lack...,...,19:sure>18:exactly[=mod],19:sure>[advmod]>18:exactly[=mod],sure>[advmod]>exactly[=mod],JJ>RB,JJ>[advmod]>RB,exactly_puddin,<NA>,<NA>,NaN,NaN
pcc_eng_09_047.0948_x0745812_11:4-5,exactly_sure,<NA>,<NA>,exactly,sure,exactly sure,pcc_eng_09_047.0948_x0745812_11,4-5,pcc_eng_09_047.0948_x0745812_11:4-5,I be not exactly sure what the big difference ...,...,04:sure>03:exactly[=mod],04:sure>[advmod]>03:exactly[=mod],sure>[advmod]>exactly[=mod],JJ>RB,JJ>[advmod]>RB,exactly_puddin,<NA>,<NA>,NaN,NaN


In [ ]:
columns = odf.columns[~odf.columns.isin(cols_by_str(
    odf, start_str=('dep_m', 'dep_n', 'dep_r', 'context')))].to_list()
columns.sort()
odf = odf.loc[:, columns]
len(odf) == len(ddf)


### Limit to `exactly` adverbs *only*

In [ ]:
odf = odf.loc[odf.adv_lemma == 'exactly', :]
odf.loc[odf.corpus.str.endswith('puddin'), 'corpus_group'] = 'puddin'
odf.loc[odf.corpus.str.endswith(('nyt', 'apw')), 'corpus_group'] = 'news'
general_counts = show_counts(odf, ['category', 'corpus_group']).unstack(
).sort_values(('count', 'puddin'), ascending=False)  # type: ignore
general_counts


*NOTE:* 👇 *`odf` is shorter (fewer rows) than original loaded hits because adverbs other than `'exactly'` have been dropped.*

In [ ]:
len(odf) < len(ddf)


### Add `conllu_id` and drop unused columns

In [ ]:
odf = odf.assign(conllu_id=odf.sent_id.str.rsplit('_', 2).str.get(0).str.split(
    '.').str.get(0).astype('string').astype('category'))  # type: ignore
tdf = odf[cols_by_str(odf, end_str=('lemma', 'id', 'text', 'window',
                      'category', 'Pol')) + cols_by_str(odf, start_str=('corpus', 'lemma'))]
print(f'Total "exactly" hits for all patterns: {len(tdf)}')
summary_tdf = summarize_text_cols(tdf)
summary_tdf


## Identify $PosPol$ and $NegPol$ contexts

### Option A
bare collocation tokens (`advadj.all-RB-JJs` pattern match) which do not appear as matches for any other pattern match (i.e. $NegPol$ contexts).

*That is, the `colloc_id` (unique `ADV` & `ADJ` nodes in unique sentence tokens) is not duplicated.*

```{python}
tdfp_a = tdf.loc[(tdf.category=='advadj') & (~tdf.duplicated(subset='colloc_id', keep=False)), :]
```

### Option B
categorize $NegPol$ set first (`tdfn`), then compute complement of that (i.e. $ALL - NegPol$)

```{python}
tdfn = tdf.loc[tdf.neg_lemma!='_', :]
tdfp = tdf.loc[~tdf.colloc_id.isin(tdfn.colloc_id), :]
```

### Options A and B are identical

`all(tdfp_a.index == tdfp_b.index)` evaluates as true

So, since $NegPol$ is more directly defined, and has to be separated out anyway, it's simpler to just get the "complement", (`tdfp_b` method)

In [ ]:
tdfn = tdf.loc[tdf.neg_lemma != '_', :]
summarize_text_cols(tdfn)


In [ ]:
tdfp = tdf.loc[~tdf.colloc_id.isin(tdfn.colloc_id), :]
summarize_text_cols(tdfp)


## Assign `polarity` and recombine

In [ ]:
tdfp = tdfp.assign(polarity='positive')
tdfn = tdfn.assign(polarity='negative')
tdf_with_overlap = tdf

pol_union_df = pd.concat([tdfp, tdfn]).sort_values('colloc_id')

print(
    f'Total bare `exactly ADJ` collocations: {odf.category.value_counts()["advadj"]}')
print(
    f'Total `exactly` pattern hits (+ NegPol pattern overlap): {len(pol_union_df)}')
print(f'PosPol: {round(100*len(tdfp)/len(pol_union_df))}% : {len(tdfp)} hits')
print(
    f'NegPol: {str(round(100*len(tdfn)/len(pol_union_df))).zfill(2)}% : {len(tdfn)} hits')


### Pickle Pos+Neg Dataframe

In [ ]:
pol_union_df.to_pickle(
    EXACTLY_OUT/'all-exactly-hits_text+polarity.pkl.gz')


## Frequency by Polarity

### Simple Frequency Comparison

In [ ]:
top_200 = show_counts(pol_union_df, ['polarity', 'adj_lemma']).head(200)
top_200


In [ ]:
def snake_to_camel(snake: str):
    return ''.join([w.capitalize() for w in snake.split('_')])


def save_for_ucs(df, col_1: str, col_2: str,
                 output_dir: Path = None,
                 ucs_path: Path = None,
                 filter: dict = None,
                 ):
    if output_dir and not ucs_path:
        ucs_path = (output_dir / 'ucs_format' /
                    f'{snake_to_camel(col_1)}{snake_to_camel(col_2)}.tsv')
    if filter:
        for col, val in filter.items():
            df = df.loc[df[col] == val, :]
            if ucs_path:
                ucs_path = ucs_path.with_stem(
                    f"{ucs_path.stem}_{snake_to_camel(col)}{val.upper()}")
    if ucs_path:
        print(f'output path: {ucs_path}')
        counts = show_counts(df, [col_1, col_2]).reset_index()[
            ['count', col_1, col_2]]
        counts.to_csv(ucs_path, encoding='utf8',
                      sep='\t', header=False, index=False)

    return counts

In [ ]:
save_for_ucs(pol_union_df, 'polarity', 'adj_lemma', filter={'corpus_group':'news'}, output_dir = EXACTLY_OUT)

In [ ]:
from utils.visualize import heatmap
from utils.dataframes import transform_counts

In [ ]:
spread_top_200 = top_200.unstack(level='polarity', fill_value=0)
spread_top_200.sort_values(('count', 'negative'),
                           ascending=False).head(30)  # type: ignore


In [ ]:
heatmap(transform_counts(spread_top_200.sort_values(('count', 'negative'),
                           ascending=False).head(30)))

In [ ]:
print(
    spread_top_200.sort_values(('count', 'positive'),
                               ascending=False)
    .head(10).to_markdown(floatfmt=',.0f'))  # type: ignore

### Crosstabulate adjective by context polarity

In [ ]:

def compute_ratios(freq_dist:pd.DataFrame, dimensions:list, abbr_len:int=3):
    for dimension in dimensions:
        freq_dist[f'ratio_{dimension[:abbr_len]}'] = (
            freq_dist[dimension]/freq_dist.TOTAL).round(3)


def add_bins(freq_dist):
    for ratio_col in freq_dist.copy().filter(like='ratio'):
        dim_ratio = freq_dist[ratio_col]
        bin_col = pd.to_numeric(dim_ratio.round(1), downcast='float')
        bin_col = bin_col.astype('category')
        freq_dist[ratio_col.replace('ratio', 'bin')] = bin_col


def compute_meta_cols(df: pd.DataFrame,
                      obs_col: str = 'adj_lemma',
                      dim_col: str = 'polarity',
                      dim_abbr_len: int = 3):
    dimensions = list(df[dim_col].unique())
    freq_dist = pd.crosstab(df[obs_col],
                            df[dim_col],
                            margins=True, margins_name='TOTAL')
    freq_dist = freq_dist[['TOTAL'] + dimensions]
    compute_ratios(freq_dist, dimensions, dim_abbr_len)
    add_bins(freq_dist)

    # cols = freq_dist.columns.to_list()
    # freq_dist = freq_dist[[cols.pop(cols.index('TOTAL'))] + cols]
    return freq_dist.sort_values('TOTAL', ascending=False)



In [ ]:
freq_dist = compute_meta_cols(pol_union_df)
freq_dist

In [ ]:
compute_meta_cols(pol_union_df, obs_col='polarity', dim_col='corpus_group')

In [ ]:
compute_meta_cols(pol_union_df, obs_col='polarity', dim_col='corpus_group', dim_abbr_len=4)

In [ ]:
compute_meta_cols(pol_union_df, dim_col='polarity', obs_col='corpus_group', dim_abbr_len=3)

In [ ]:
compute_meta_cols(pol_union_df, obs_col='corpus', dim_col='corpus_group', dim_abbr_len=4)

In [ ]:
freq_thresh5 = freq_dist.loc[freq_dist.TOTAL >= 5, :]
freq_thresh5 = freq_thresh5.sort_values(
    ['bin_neg', 'TOTAL', 'ratio_neg'], ascending=False)
freq_thresh5.to_csv(
    EXACTLY_OUT / 'freq_thresh5.csv')
freq_thresh5


In [ ]:
freq_dist.ratio_neg.plot(kind='hist')

In [ ]:
freq_dist.loc[freq_dist.index!='TOTAL', ['negative', 'positive']].plot(x='negative', y ='positive', kind='scatter')

In [ ]:
freq_dist.loc[freq_dist.index!='TOTAL', ['negative', 'positive']].plot(x='negative', y ='positive', kind='density')

In [ ]:
freq_dist.ratio_pos.plot(kind='hist')

In [ ]:
freq_thresh100 = freq_dist.loc[freq_dist.TOTAL >= 100, :]
freq_thresh100 = freq_thresh100.sort_values(
    ['bin_neg', 'TOTAL', 'ratio_neg'], ascending=False)
freq_thresh100.to_csv(
    '/share/compling/projects/sanpi/notebooks/exactly_out/freq_thresh100.csv')
freq_thresh100


In [ ]:
freq_thresh200 = freq_dist.loc[freq_dist.TOTAL >= 200, :]
freq_thresh200 = freq_thresh200.sort_values(
    ['bin_neg', 'TOTAL', 'ratio_neg'], ascending=False)
freq_thresh200.to_csv(EXACTLY_OUT / 'freq_thresh200.csv')
freq_thresh200


## Save $PosPol$ text data

Since $PosPol$ is defined as the complement of $NegPol$, accuracy relies on $NegPol$ catching all relevant cases.

To ensure pattern specifications are sufficiently inclusive, all sentences with supposedly positive polarity
should be manually inspected for any errant (uncaught) negative lemmas, as identified in the $NegPol$ pattern specifications.

```{ocaml}
NEG [lemma="hardly"|"scarcely"|"never"|"rarely"|"barely"|"seldom"|"no"|"nothing"|"none"|"nobody"|"neither"|"without"|"few"|"nor"];  
```

- [x] create simplified output of $PosPol$/`tdfp` sentence text data, with necessary identifiers
- [x] save as csv
- [ ] grep `pos_sentences.csv` for each neg lemma:\
  *There should not be any negative lemmas an `exactly JJ` collocation in its scope.*


In [ ]:
select_cols = ['adj_lemma', 'text_window', 'sent_text',
               'sent_id', 'conllu_id', 'corpus']
pos_text_info = tdfp.loc[:, select_cols]
pos_text_info = pos_text_info.assign(
    adj_neg_ratio=pos_text_info.adj_lemma.apply(lambda a: freq_dist.loc[a, 'ratio_neg'] if a in freq_dist.index else None))  # type: ignore
pos_text_info.sort_values(['adj_neg_ratio', 'conllu_id'], ascending=False)
pos_text_info.to_csv(EXACTLY_OUT / 'pos_sentences.csv')

# tdfp.adj_lemma[~tdfp.adj_lemma.isin(freq_dist.index)].value_counts()
# TODO?? why does it say there is a mismatch between the crosstab and pospol adj set?
# ^ 🤔 probably something to do with categorical dtype


In [ ]:
freq_thresh5.describe().round(2)

In [ ]:
# pol_union_thresh5 = pol_union_df.loc[pol_union_df.adj_lemma]
adj_x_cat = (pd.crosstab(index=pol_union_df.adj_lemma, columns=pol_union_df.category, 
                         margins=True, margins_name='TOTAL', normalize='index')
             .rename(columns={'advadj':'PosPol', 'contig':'contig_NegPol', 
                              'raised': 'raised_NegPol', 'scoped':'scoped_NegPol'})
             .round(3))
adj_x_cat.loc[adj_x_cat.index.isin(freq_thresh200.index), :]

In [ ]:
adj_x_neg = pd.crosstab(index=pol_union_df.adj_lemma,
                        columns=pol_union_df.neg_lemma, margins=True)
adj_x_neg.nlargest(20, columns=['All'])
